In [51]:
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')

from scipy import stats
from scipy.stats import norm, skew #for some statistics

pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points
pd.set_option('display.max_columns', 500)

from subprocess import check_output

In [52]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error

In [53]:
train_data_path = "../../datasets/house/train.csv";
test_data_path = "../../datasets/house/test.csv";

In [29]:
print(check_output(["ls","../../datasets/house"]).decode("utf8"))

data_description.txt
test.csv
train.csv



In [30]:
train = pd.read_csv(train_data_path)
test = pd.read_csv(test_data_path)

In [31]:
train = train.drop('Id',axis=1)
test_ids = test['Id']
test = test.drop('Id',axis=1)

In [32]:
train.shape

(1460, 80)

In [33]:
# train.columns

In [34]:
# 学習データの欠損状況
def cndNull(df):
    print(df.isnull().sum()[df.isnull().sum()>0])#.sort_values(ascending=False)


In [35]:
train.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.000,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.000,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.000,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.000,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.000,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.000,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.000,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.000,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.000,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.000,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.000,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.000,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.000,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.000,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.000,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [11]:
# sns.scatterplot(x="LotFrontage", y="SalePrice", data=train)

In [12]:
# sns.distplot(train['LotFrontage'].dropna() , fit=norm);

In [13]:
# #We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
train["LogSalePrice"] = np.log1p(train["SalePrice"])

# #Check the new distribution 
# sns.distplot(train['SalePrice'] , fit=norm);

In [14]:
# #Get also the QQ-plot
# fig = plt.figure()
# res = stats.probplot(train['LogSalePrice'], plot=plt)
# plt.show()

In [15]:
#欠損値除外関数
def drop_df_na(df):
    df['PoolQC'].fillna(0,inplace=True)#poolないので
    df['Electrical'].fillna('None',inplace=True)#電気ないので
    
    df['GarageCars'].fillna(0,inplace=True)#ガレージないので
    df['GarageArea'].fillna(0,inplace=True)#ガレージないので
    df['GarageType'].fillna('None',inplace=True)#ガレージないので
    df['GarageFinish'].fillna('None',inplace=True)#ガレージないので
    df['GarageQual'].fillna('None',inplace=True)#ガレージないのde  
    df['GarageCond'].fillna('None',inplace=True)#ガレージないのde
    df['GarageYrBlt'].fillna(df['YearBuilt'],inplace=True)#ガレージないので築年数で補填
    
    df['BsmtFinSF1'].fillna(0,inplace=True)#地下室ないのde
    df['BsmtFinSF2'].fillna(0,inplace=True)#地下室ないのde
    df['BsmtFullBath'].fillna(0,inplace=True)#地下室ないのde
    df['BsmtHalfBath'].fillna(0,inplace=True)#地下室ないのde
    df['BsmtUnfSF'].fillna(0,inplace=True)#地下室ないのde
    df['TotalBsmtSF'].fillna(0,inplace=True)#地下室ないのde
    df['BsmtFinType2'].fillna('None',inplace=True)#地下室ないのde
    df['BsmtFinType1'].fillna('None',inplace=True)#地下室ないのde
    df['BsmtExposure'].fillna('None',inplace=True)#地下室ないのde
    df['BsmtCond'].fillna('None',inplace=True)#地下室ないのde
    df['BsmtQual'].fillna('None',inplace=True)#地下室ないのde
    
    df['MiscFeature'].fillna('None',inplace=True)#特別機能ないのde
    
    df['Alley'].fillna('None',inplace=True)#小道ないのde
    
 
    df['MasVnrType'].fillna('None',inplace=True)#単純にレンガの壁ない。
    df['MasVnrArea'].fillna(0,inplace=True)#単純にレンガの壁ない。
    
    df[df['Fence'].isnull()==False&df['FireplaceQu'].isnull()].shape#Fence1欠損が1179で合計すると全体
    df['FireplaceQu'].fillna('None',inplace=True)#フェンスではないのde
    df['Fence'].fillna('None',inplace=True)#フェンすがないのde
    
    # sns.scatterplot(x="LotFrontage", y="SalePrice", data=df)
    df['LotFrontage'].fillna(df['LotFrontage'].median(),inplace=True)#outage discard→median
    
#     sns.countplot(x='KitchenQual', data=test)
    df['KitchenQual'].fillna('TA',inplace=True)#該当数件で単純な欠損。TAが多いので
    
#     sns.countplot(x='Functional', data=test)
    df['Functional'].fillna('Typ',inplace=True)#該当数件で単純な欠損。Typが多いので
    
#     sns.countplot(x='Utilities', data=test)
    df['Utilities'].fillna('AllPub',inplace=True)#ライフラインない残念な家はない。ALｌpubがすべて

#     sns.countplot(x='Exterior2nd', data=test)
    df['Exterior2nd'].fillna('VinylSd',inplace=True)#該当数件で単純な欠損。VinylSd が最多@test

#     sns.countplot(x='Exterior1st', data=test)
    df['Exterior1st'].fillna('VinylSd',inplace=True)#該当数件で単純な欠損。VinylSd が最多@test
    
#     sns.countplot(x='MSZoning', data=test)
    df['MSZoning'].fillna('RL',inplace=True)#該当数件で単純な欠損。RL がすべて@test
    
    df['SaleType'].fillna('WD',inplace=True)#該当数件で単純な欠損。WD がすべて@test
    
    cndNull(df)
    
    return df

In [16]:
#特徴量エンジニアリング
def eng_df(df):
    df['FlrSF'] = df['TotalBsmtSF']+df['1stFlrSF'] + df['2ndFlrSF'] #特徴量でフロアが大事らしいって結果になったので
    df['MSSubClass'] = df['MSSubClass'].apply(str)#カテゴリカル
    df['YrSold'] = df['YrSold'].astype(str)#カテゴリカル
    df['MoSold'] = df['MoSold'].astype(str)#カテゴリカル
    
    return df

### 特徴量エンジニアリング

In [17]:
test= drop_df_na(test) #除外または補完
test= eng_df(test) #特徴量生成

Series([], dtype: int64)


In [18]:
train= drop_df_na(train)
train= eng_df(train)

Series([], dtype: int64)


In [58]:
ns.scatterplot(data=tips, x='total_bill', y='tip', hue='smoker')

NameError: name 'ns' is not defined

In [ ]:
sns.relplot(data=tips, kind='scatter', x='total_bill', y='tip', col='smoker')

In [ ]:
sns.boxplot(data=titanic, x='pclass', y='age')

In [ ]:
sns.countplot(data=titanic, x='pclass')

### 詳細分析

In [44]:
clf = {}

#### kmeans

In [38]:
from sklearn.cluster import KMeans

In [45]:
clf['kmeans'] = KMeans(
    n_clusters=8,
    init='k-means++',
    n_init=10,
    max_iter=300,
    tol=0.0001,
    precompute_distances='auto',
    verbose=0,
    random_state=None,
    copy_x=True,
    n_jobs=4
)

In [47]:
# clf['kmeans'].fit_predict(train)

#### CART

In [48]:
from sklearn.tree  import DecisionTreeClassifier

In [57]:
clf['cart'] =\
    DecisionTreeClassifier(
        criterion='gini',
        splitter='best', 
        max_depth=None, 
        min_samples_split=2,
        min_samples_leaf=1,
        min_weight_fraction_leaf=0.0,
        max_features=None,
        random_state=None,
        max_leaf_nodes=None,
        class_weight=None,
        presort=False
    )

In [ ]:
clf['cart'].fit(train, data['x3'])

### 予測実行

In [19]:
# dfs

In [20]:
# 半分パクリ
dfs = pd.concat([train,test]) 
cols = ("MSZoning","LotFrontage","Street","Alley","LotShape","LandContour","Utilities","LotConfig","LandSlope","Neighborhood","Condition1","Condition2","BldgType","HouseStyle","RoofStyle","RoofMatl","Exterior1st","Exterior2nd","MasVnrType","MasVnrArea","ExterQual","ExterCond","Foundation","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2","Heating","HeatingQC","CentralAir","Electrical","KitchenQual","Functional","FireplaceQu","GarageType","GarageYrBlt","GarageFinish","GarageQual","GarageCond","PavedDrive","PoolQC","Fence","MiscFeature","SaleType","SaleCondition")
# process columns, apply LabelEncoder to categorical features
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(dfs[c].values)) 

    dfs[c] = lbl.transform(list(dfs[c].values))
    
dfs = dfs.drop('SalePrice',axis=1)
#ここだけ特徴量捜査
dfs = dfs.drop(["MiscVal","MiscFeature","PoolQC","BsmtFinType2","Fence","LotConfig","BldgType","Utilities","HouseStyle","Heating","3SsnPorch","LowQualFinSF"],axis=1)#重要度低い

test = dfs[dfs['LogSalePrice'].isnull()==True]
test = test.drop('LogSalePrice',axis=1)

train = dfs[dfs['LogSalePrice'].isnull()==False]

/Users/macico/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [ ]:
#データセット生成
x_train, x_test, y_train, y_test = train_test_split(train.drop('LogSalePrice',axis=1), train['LogSalePrice'], test_size=0.25)




In [21]:
# tuned_parameters = [
#     {'n_estimators': [1, 10, 100, 1000],
#      'learning_rate': [0.1,0.05,0.01],
#      'max_depth':[3,5,10,15]}]
tuned_parameters = [
    {'n_estimators': [1000,2000,3000],#
     'learning_rate': [0.1,0.05,0.01],
     'min_samples_leaf':[5,10,15,20], 
     'min_samples_split':[5,10,15,20],
     'max_depth':[3,6,9]} #深さ
    ]
score = 'f1'
clf = GridSearchCV(
    GradientBoostingRegressor(), # 識別器
    tuned_parameters, # 最適化したいパラメータセット 
    cv=5, # 交差検定の回数
    verbose=1,
    n_jobs=4#並列
    ) # モデルの評価関数の指定
clf.fit(x_train, y_train)



Fitting 5 folds for each of 432 candidates, totalling 2160 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.2min


KeyboardInterrupt: 

In [ ]:
y_true, y_pred = y_test, clf.best_estimator_.predict(x_test)#best_estimator_は不要
np.sqrt(mean_squared_error(y_true, y_pred))

In [ ]:
name_df = pd.DataFrame(test.columns,columns=['name'])

In [ ]:
score_df = pd.DataFrame(clf.best_estimator_.feature_importances_,columns=['score'])

In [ ]:
scores_df = pd.concat([name_df,score_df],axis=1)

In [ ]:
scores_df.sort_values(by=['score'],ascending=False,inplace=True)

In [322]:
scores_df[scores_df['score']==0.0].name

56      RoofMatl
55      PoolArea
61        Street
7     BsmtFinSF2
15    Condition2
41     LandSlope
Name: name, dtype: object

In [323]:
pred =  clf.best_estimator_.predict(test)

In [324]:
result_df = (pd.DataFrame((np.exp(pred - 1))))
result_df = result_df.reset_index()
result_df = result_df.rename(columns={'LogSalePrice':'SalePrice','index':'Id'})
result_df.to_csv('result_train.csv',index=False)

In [325]:
result_df= (pd.DataFrame((np.exp(pred - 1)),columns=['SalePrice']))
result_df['Id'] = test_ids
result_df.to_csv('result.csv',index=False)

In [1]:
import plaidml.keras
plaidml.keras.install_backend()
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13234833591667567201]

In [50]:
train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.000,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.000,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.000,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.000,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.000,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.000,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.000,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.000,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.000,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.000,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.000,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.000,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.000,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.000,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.000,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,50,RL,85.000,14115,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0.000,TA,TA,Wood,Gd,TA,No,GLQ,732,Unf,0,64,796,GasA,Ex,Y,SBrkr,796,566,0,1362,1,0,1,1,1,1,TA,5,Typ,0,NaN,Attchd,1993.000,Unf,2,480,TA,TA,Y,40,30,0,320,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,20,RL,75.000,10084,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186.000,Gd,TA,PConc,Ex,TA,Av,GLQ,1369,Unf,0,317,1686,GasA,Ex,Y,SBrkr,1694,0,0,1694,1,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2004.000,RFn,2,636,TA,TA,Y,255,57,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,60,RL,nan,10382,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,Stone,240.000,TA,TA,CBlock,Gd,TA,Mn,ALQ,859,BLQ,32,216,1107,GasA,Ex,Y,SBrkr,1107,983,0,2090,1,0,2,1,3,1,TA,7,Typ,2,TA,Attchd,1973.000,RFn,2,484,TA,TA,Y,235,204,228,0,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,50,RM,51.000,6120,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,CompShg,BrkFace,Wd Shng,None,0.000,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,952,952,GasA,Gd,Y,FuseF,1022,752,0,1774,0,0,2,0,2,2,TA,8,Min1,2,TA,Detchd,1931.000,Unf,2,468,Fa,TA,Y,90,0,205,0,0,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,190,RL,50.000,7420,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,Artery,2fmCon,1.5Unf,5,6,1939,1950,Gable,CompShg,MetalSd,MetalSd,None,0.000,TA,TA,BrkTil,TA